In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler


In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass
import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager

In [ ]:
dm = DataManager()
feed = dm.datafeed

In [ ]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2011, 1, 1))

In [ ]:
chain_values    

In [ ]:
fut = chain.get_contract(datetime(2012, 1, 1))
fut2 = chain.get_contract(datetime(2012, 1, 1), offset=1)

In [ ]:
series1 = dm.series_get(fut)
series2 = dm.series_get(fut2)

In [ ]:
series  = dm.series_get(fut)

In [ ]:
series.c.plot()

In [ ]:
prev_series, position1  = compress_daily(DataFrameGetter(series1), fut)
new_series, position2  = compress_daily(DataFrameGetter(series2), fut2)

In [ ]:
series1.to_csv('fut_series_for_contfut1.csv.gz', compression='gzip')

In [ ]:
series2.to_csv('fut_series_for_contfut2.csv.gz', compression='gzip')

In [ ]:
prev_series

In [ ]:
new_series.c.sum()

In [ ]:
prev_prices = prev_series.loc[prev_series.index[-1]]
new_prices = new_series.loc[prev_series.index[-1]]

In [ ]:
prev_prices

In [ ]:
new_prices

In [ ]:
fut_offset = new_prices['exec'] - prev_prices['exec']

In [ ]:
fut_offset

In [ ]:
new_series[['o','h','l', 'c', 'exec']] -= fut_offset

In [ ]:
new_series = new_series[new_series.index > prev_series.index[-1]]

In [ ]:
pd.concat([prev_series, new_series])

In [ ]:
def merge(prev_series, new_series):
    try:
        prev_prices = prev_series.loc[prev_series.index[-1]]
        new_prices = new_series.loc[prev_series.index[-1]]
        # Calculating futures rollover factor
        fut_offset = new_prices['exec'] - prev_prices['exec']
    except KeyError:
        fut_offset = 0.0
        
    new_series[['o','h','l', 'c', 'exec']] -= fut_offset
    new_series = new_series[new_series.index > prev_series.index[-1]]
    return pd.concat([prev_series, new_series])

In [ ]:
%timeit compress_daily(DataFrameGetter(series), fut)

In [ ]:
prev_series, position1  = compress_daily(DataFrameGetter(series), fut)
new_series, position2  = compress_daily(DataFrameGetter(series2), fut2)

In [ ]:
%lprun -f merge merge(prev_series, new_series)

In [ ]:
%timeit merge(prev_series, new_series)

In [ ]:
position1.at[position1.index[-1], 'qty'] = 0

In [ ]:
position1

In [ ]:
position2

In [ ]:
%%timeit
pd.merge(position2.reset_index(), position1.reset_index(), how='outer').sort_values('date').set_index(['date', 'asset'])

In [ ]:
position1.index.get_level_values('date')

In [ ]:
from tmqrfeed.quotes.quote_contfut import QuoteContFut

In [ ]:
qcont_fut = QuoteContFut('US.CL', datamanager=dm, timeframe='D')

In [ ]:
contfut_df, contfut_position = qcont_fut.build()

In [ ]:
contfut_df.exec.plot()

In [ ]:
%lprun -m tmqrfeed.position qcont_fut.build()

In [ ]:
%timeit qcont_fut.build()

# Position caching benchmarks

In [ ]:
p = contfut_position

In [ ]:
from tmqrfeed.costs import Costs

In [ ]:
dm.costs_set("US", Costs(0, 0))

In [ ]:
p._position

In [ ]:
pos_df = p.get_pnl_series()

In [ ]:
pos_df.equity_execution.plot()

In [ ]:
contfut_df.exec.plot()
